In [1]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to videos collection

In [ ]:
videos = database['videos']

#### Drop videos with no audio description info in audio descriptions collection

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'modified_audio_descriptions',     # audio_descriptions, if modified_audio_descriptions coll. not created
            'localField': 'audio_descriptions', 
            'foreignField': '_id', 
            'as': 'videos_ad'
        }
    }, {
        '$addFields': {
            'audio_descriptions': '$videos_ad._id'
        }
    }, {
        '$project': {
            'videos_ad': 0
        }
    }, {
        '$out': 'modified_videos'   # New collection to avoid any changes/issues in current videos collection
    }
]

videos.aggregate(pipeline)